In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# We need to join the upper directory in order to access the local modules
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import itertools
import json
import logging

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import seaborn as sns
import scipy

from scipy.ndimage import uniform_filter1d
from scipy.io import loadmat, savemat

In [ ]:
plt.rcParams["font.family"] = "Linux Libertine O"
plt.rcParams["font.size"] = 10

In [ ]:
from pytdoa.mlat import exact, lls, nlls
from pytdoa import brutefoptim, nonlinoptim, linoptim
from pytdoa.geometry import generate_heatmap, generate_hyperbola
from pytdoa.geodesy import SPEED_OF_LIGHT  # noqa: E402

In [ ]:
# Reading the mat files
with open("comnets_data/gnbPositions.json") as gnb_file:
    data = json.load(gnb_file)

gNBs = np.array([data["gnb0"], data["gnb1"], data["gnb2"], data["gnb3"]])

with open("comnets_data/uePositions.json") as ue_file:
    data = json.load(ue_file)

UEs = np.array(
    [data["ue0"], data["ue1"], data["ue2"], data["ue3"], data["ue4"], data["ue5"]]
)
UEs

## Playing a bit with each of the positions (there are 6 in total)

In [ ]:
POSITION = 0
FREQUENCY_MHZ = 100  # MHz
FREQUENCY = FREQUENCY_MHZ * 1e6

In [ ]:
toa_measurements = pd.read_csv(
    f"comnets_data/experiments/exp{POSITION}/{FREQUENCY_MHZ}mhz.csv"
)
toa_measurements.drop(
    [
        "Unnamed: 0",
    ],
    axis=1,
    inplace=True,
)
toa_measurements.head(7)

In [ ]:
if FREQUENCY_MHZ == 100:
    if POSITION == 4:
        toa_measurements = toa_measurements.drop(["gNB0", "P0"], axis=1)

    toa_filt = toa_measurements.loc[~((toa_measurements == float("-inf")).any(axis=1))]
    toa_filt.reset_index(drop=True, inplace=True)

    if POSITION == 1:
        toa_filt = toa_filt.loc[154:]  # Only for Position 1

elif FREQUENCY_MHZ == 40:
    if POSITION != 2:
        toa_measurements = toa_measurements.drop(["gNB0", "P0"], axis=1)
    else:
        toa_measurements = toa_measurements.drop(["gNB1", "P1"], axis=1)

    toa_filt = toa_measurements.loc[~((toa_measurements == float("-inf")).any(axis=1))]
    toa_filt.reset_index(drop=True, inplace=True)

else:
    raise RuntimeError("Unrecorded center frequency!")

toa_filt

## Offset exploration

In [ ]:
# Let's plot a bit the drift
fig, ax = plt.subplots(figsize=(7, 5))

ax.scatter(x=np.arange(len(toa_filt)), y=toa_filt["gNB0"])
ax.scatter(x=np.arange(len(toa_filt)), y=toa_filt["gNB2"])

ax.legend(["gNB0", "gNB1"])

ax.set_xlim([0, len(toa_filt)])

ax.set_title(f"Position {POSITION} - Bandwidth: {FREQUENCY_MHZ} MHz")
ax.set_xlabel("Experiment Number")
ax.set_ylabel("Measured ToA (samples)")

In [ ]:
from util import make_grid

In [ ]:
plt.rcParams["font.family"] = "Linux Libertine O"
plt.rcParams["font.size"] = 10

(fig, ax) = make_grid(100, plot_inf=True)
plt.show()

## Getting some experiments on

In [ ]:
# Calculating the theoretical values
theoretical_toa = np.linalg.norm(UEs[POSITION] - gNBs, axis=1)
theoretical_tdoa_mat = theoretical_toa.reshape((-1, 1)) - theoretical_toa
theoretical_tdoa = theoretical_tdoa_mat[np.triu_indices(4, k=1)]
theoretical_tdoa

In [ ]:
EXPERIMENT = 1
FILTER_WINDOW = 10

#
gNB_toas = toa_filt[["gNB0", "gNB1", "gNB2", "gNB3"]].to_numpy()
triu_indices = np.triu_indices(4, k=1)
gNB_tdoas = (
    gNB_toas.reshape((gNB_toas.shape[0], gNB_toas.shape[1], 1))
    - gNB_toas.reshape((gNB_toas.shape[0], 1, gNB_toas.shape[1]))
)[:, triu_indices[0], triu_indices[1]]
selected_tdoa = gNB_tdoas[EXPERIMENT, :]
selected_tdoa = selected_tdoa / FREQUENCY * SPEED_OF_LIGHT
selected_tdoa

In [ ]:
# Let's plot a bit the drift
fig, ax = plt.subplots(figsize=(7, 5))

ax.scatter(x=np.arange(len(gNB_tdoas)), y=gNB_tdoas[:, 1])

ax.legend(["gNB0-gNB1"])

ax.set_ylim([-5, 5])
ax.set_xlim([0, len(toa_filt)])

ax.set_title(f"Position {POSITION} - Bandwidth: {FREQUENCY_MHZ} MHz")
ax.set_xlabel("Experiment Number")
ax.set_ylabel("Measured ToA (samples)")

In [ ]:
linpos = linoptim(gNBs, selected_tdoa[0:4], input_type="xyz").squeeze()
print(f"Calculated Position - x: {linpos[0]:.4f} y: {linpos[1]:.4f}")
print(f"Real Position - x: {UEs[POSITION,0]:.4f} y: {UEs[POSITION,1]:.4f}")

In [ ]:
combination_range = itertools.combinations(np.arange(len(gNBs)), 2)
combinations = np.fromiter(combination_range, dtype=np.dtype((int, 2)))

position_nlls = nonlinoptim(
    gNBs,
    selected_tdoa,
    combinations,
    num_dim=2,
    p0=np.mean(gNBs, axis=0),
    input_type="xyz",
    method="SLSQP",
)
position_nlls

## Looping over all values

In [ ]:
position_lls = 50 * np.ones((len(toa_filt), 2))
position_nls = 50 * np.ones((len(toa_filt), 2))

filter_w = 1
if FREQUENCY_MHZ == 100:
    if POSITION == 4:
        n = 3
        gNB_toas = uniform_filter1d(
            toa_filt[["gNB1", "gNB2", "gNB3"]].to_numpy(), filter_w, mode="mirror"
        )
        gNB_loop = gNBs[1:, :]
    else:
        n = 4
        gNB_toas = uniform_filter1d(
            toa_filt[["gNB0", "gNB1", "gNB2", "gNB3"]].to_numpy(),
            filter_w,
            mode="mirror",
        )
        gNB_loop = gNBs
if FREQUENCY_MHZ == 40:
    if POSITION == 2:
        gNB_toas = uniform_filter1d(
            toa_filt[["gNB0", "gNB2", "gNB3"]].to_numpy(), filter_w, mode="mirror"
        )
        gNB_loop = gNBs
    else:
        gNB_toas = uniform_filter1d(
            toa_filt[["gNB1", "gNB2", "gNB3"]].to_numpy(), filter_w, mode="mirror"
        )
        gNB_loop = gNBs

combination_range = itertools.combinations(np.arange(len(gNB_loop)), 2)
combinations = np.fromiter(combination_range, dtype=np.dtype((int, 2)))
for i in range(len(toa_filt)):
    gNB_toas_selected = gNB_toas[i, :]
    gNB_tdoas_selected_mat = gNB_toas_selected.reshape((-1, 1)) - gNB_toas_selected
    selected_tdoa = gNB_tdoas_selected_mat[np.triu_indices(n, k=1)]
    selected_tdoa = selected_tdoa / FREQUENCY * SPEED_OF_LIGHT

    position_lls[i, :] = linoptim(
        gNB_loop, selected_tdoa[0:n], input_type="xyz"
    ).squeeze()
    position_nls[i, :] = nonlinoptim(
        gNB_loop,
        selected_tdoa,
        combinations,
        num_dim=2,
        p0=np.mean(gNB_loop, axis=0),
        input_type="xyz",
        method="SLSQP",
    ).squeeze()

In [ ]:
error_lls = np.linalg.norm((UEs[POSITION, :] - position_lls), axis=1)
error_nls = np.linalg.norm((UEs[POSITION, :] - position_nls), axis=1)

In [ ]:
plt.rcParams["font.family"] = "Linux Libertine O"
plt.rcParams["font.size"] = 20

In [ ]:
f, ax = plt.subplots(figsize=(8, 4))

ax.ecdf(error_lls)
ax.ecdf(error_nls)

ax.set_xlim([0, 5])
ax.grid()

ax.set_title(f"Position {POSITION} - Bandwidth: {FREQUENCY_MHZ} MHz")
ax.set_xlabel("Error (m)")
ax.legend(["Linear", "Non-Linear/Brute-Force"])